In [1]:
import os
import tensorflow as tf
import argparse
from tensorflow.python.keras.callbacks import Callback

In [2]:
class MyModel(object):
    def train(self):
        mnist = tf.keras.datasets.mnist

        # 입력 값을 받게 추가합니다.
        parser = argparse.ArgumentParser()
        parser.add_argument('--learning_rate', required=False, type=float, default=0.01)
        parser.add_argument('--dropout_rate', required=False, type=float, default=0.2)
        # epoch 5 ~ 15
        parser.add_argument('--epoch', required=False, type=int, default=5)    
        # relu, sigmoid, softmax, tanh
        parser.add_argument('--act', required=False, type=str, default='relu')        
        # layer 1 ~ 5
        parser.add_argument('--layer', required=False, type=int, default=1) 
        
        args = parser.parse_args()

        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train, x_test = x_train / 255.0, x_test / 255.0

        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
        
        for i in range(int(args.layer)):    
            model.add(tf.keras.layers.Dense(128, activation=args.act))
            model.add(tf.keras.layers.Dropout(args.dropout_rate))
            
        model.add(tf.keras.layers.Dense(10, activation='softmax'))
        model.summary()
        
        model.compile(optimizer=tf.keras.optimizers.Adam(lr=args.learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

        model.fit(x_train, y_train,
              verbose=0,
              validation_data=(x_test, y_test),
              epochs=args.epoch,
              callbacks=[KatibMetricLog()])

In [3]:
class KatibMetricLog(Callback):
    def on_batch_end(self, batch, logs={}):
        print("batch=" + str(batch),
              "accuracy=" + str(logs.get('acc')),
              "loss=" + str(logs.get('loss')))
    def on_epoch_begin(self, epoch, logs={}):
        print("epoch " + str(epoch) + ":")
    
    def on_epoch_end(self, epoch, logs={}):
        print("Validation-accuracy=" + str(logs.get('val_acc')),
              "Validation-loss=" + str(logs.get('val_loss')))
        return

In [4]:
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils

        DOCKER_REGISTRY = 'kubeflow-registry.default.svc.cluster.local:30000'
        fairing.config.set_builder(
            'append',
            image_name='katib-job',
            base_image='kubeflow-registry.default.svc.cluster.local:30000/kubeflow-jupyterlab:tf2.3-cpu',
            registry=DOCKER_REGISTRY,
            push=True)
        # cpu 1, memory 1GiB
        fairing.config.set_deployer('job', namespace='koock')
        # python3
        #fairing.config.set_preprocessor('python', input_files=[__file__])
        fairing.config.run()
    else:
        remote_train = MyModel()
        remote_train.train()

[I 200924 11:09:43 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f9a1a5bbcf8>
[I 200924 11:09:43 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f9a66e1d0b8>
[I 200924 11:09:43 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f9a19d33f60>
[W 200924 11:09:43 append:50] Building image using Append builder...
[I 200924 11:09:43 base:107] Creating docker context: /tmp/fairing_context_p98crxz_
[I 200924 11:09:43 converted_notebook:127] Converting mnist-katib-bayesian.ipynb to mnist-katib-bayesian.py
[I 200924 11:09:43 docker_creds_:234] Loading Docker credentials for repository 'kubeflow-registry.default.svc.cluster.local:30000/kubeflow-jupyterlab:tf2.3-cpu'
[W 200924 11:09:43 append:54] Image successfully built in 0.6582023329992808s.
[W 200924 11:09:43 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/katib-jo

11493376/11490434 [==============================] - 0s 0us/step
2020-09-24 11:10:51.853052: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-09-24 11:10:51.960543: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2497120000 Hz
2020-09-24 11:10:51.960800: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559fa5a013f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-24 11:10:51.960832: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-24 11:11:28.958901: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 188160000 exceeds 10% of free system memory.
Model: "sequent

[W 200924 11:11:47 job:173] Cleaning up job fairing-job-dbg6q...
